In [5]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import os, json

/anaconda3/lib/python3.7/site-packages/matplotlib/__init__.py:1066: UserWarning: Duplicate key in file "/Users/hy/.matplotlib/matplotlibrc", line #2
  (fname, cnt))


In [6]:
pd.set_option('display.max_columns', None)

In [ ]:
# 读取data文件夹中的json数据，并转化为DataFrame
path_to_json = 'data/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

js = list()
for json_file in json_files:
    with open(path_to_json + json_file, 'r') as json_file:
        js.append(json.load(json_file))
vehicle = pd.DataFrame(js[0]['Vehicle'])
station = pd.DataFrame(js[1]['Station'])
matrix = pd.DataFrame(js[2]['Matrix'])
box = pd.DataFrame(js[3]['Bin'])

In [ ]:
# 评估最少经费的函数


In [21]:
vehicle.head()

,distance_fare,flag_down_fare,vehicle_id,vehicle_length,vehicle_weight,vehicle_width
0,0.004,888,V001,7.8,10000,2.3
1,0.004,888,V002,7.8,10000,2.3
2,0.004,888,V003,7.8,10000,2.3
3,0.004,888,V004,7.8,10000,2.3
4,0.004,888,V005,7.8,10000,2.3


In [23]:
station.head()

,loading_time,station_id,vehicle_limit
0,24,S001,8
1,19,S002,10
2,19,S003,10
3,22,S004,10
4,14,S005,8


In [24]:
box.head()

,bin_id,bin_length,bin_weight,bin_width,station
0,B00001,1.65,132,0.85,S039
1,B00002,1.65,195,0.85,S053
2,B00003,1.65,32,0.85,S001
3,B00004,1.65,118,0.85,S057
4,B00005,1.65,398,1.65,S096


In [25]:
matrix.head()

,arrival_station_id,departure_station_id,distance,time
0,S001,S001,1,1.0
1,S001,S002,1,1.0
2,S001,S003,29701,29.0
3,S001,S004,255,2.0
4,S001,S005,31108,34.0


In [26]:
# 随机运送
from random import choice

# 随机选择1辆车
car_1 = choice(vehicle['vehicle_id'])
# 随机选择1个货箱
box_1 = choice(box['bin_id'])

In [27]:
# 按照要求，建立输出结果的字典
result = dict()
result[car_1] = dict()
result[car_1]['Route'] = list(box[box['bin_id']==box_1]['station'])

In [28]:
# 装货
vehicle[vehicle['vehicle_id']==car_1]

,distance_fare,flag_down_fare,vehicle_id,vehicle_length,vehicle_weight,vehicle_width
406,0.00418,999,V407,9.6,15000,3.0


In [29]:
result

{'V407': {'Route': ['S054']}}

In [33]:
box[box['bin_id']==box_1]['bin_length']

2759    1.28
Name: bin_length, dtype: float64

In [35]:
box[box['bin_id']==box_1]['bin_width']

2759    1.05
Name: bin_width, dtype: float64

In [34]:
# 采用BL二维装箱算法，先将第一个物体放置于右下角，然后向上移动优先，直到不能向上移动，向左移动
class BLpacking():
    
    # 初始化卡车的长、宽、高，并初始化装载结果
    def __init__(self, truck_id, truck_length, truck_width, truck_weight):
        self.truck_id = truck_id
        self.truck_length = truck_length
        self.truck_width = truck_width
        self.truck_weight = truck_weight
        self.packed_output = dict()
        self.packed_output[self.truck_id] = dict()
        self.packed_output['box_exit'] = False
        self.packed_output['sum_weight'] = 0
    
    # 装箱方法
    def packing(self, box_id, box_length, box_width, box_weight):
        self.box_id = box_id
        self.box_length = box_length
        self.box_width = box_width
        self.box_weight = box_weight
        self.pack_tf = False
        
        # 判断1:如果货物长宽大于货车长宽，或者货物重量超过剩余可装载重量，返回False以及之前的装载结果
        if((self.box_length>self.truck_length)|(self.box_width>self.truck_width)|(self.box_weight>(self.truck_weight-self.packed_output['sum_weight']))):
            return self.pack_tf, self.packed_output
        # 判断2:如果卡车上的货物为空，将这个标志值改为True，并且将该货物装载到卡车上，记录装货的坐标，并返回结果
        elif self.packed_output['box_exit'] == False:
            self.packed_output['box_exit'] = True
            self.packed_output[self.truck_id][box_id] = np.zeros((4,2)) #初始化一个4行2列的数组，并利用长宽赋值数组，记录位置
            self.packed_output[self.truck_id][box_id][1][1] = self.box_width 
            self.packed_output[self.truck_id][box_id][2][1] = self.box_width
            self.packed_output[self.truck_id][box_id][2][0] = self.box_length
            self.packed_output[self.truck_id][box_id][3][0] = self.box_length
            
            self.packed_output['sum_weight'] += self.box_weight
            self.pack_tf = True
            return self.pack_tf, self.packed_output
        # 否则，将货物置于右下角最底端
        else:  
            temp = np.zeros((4,2))
            temp[:,0] = self.truck_length
            temp[:,1] = self.truck_width
            temp[:2,0] = self.truck_length - self.box_length
            temp[0,1] = self.truck_width - self.box_width
            temp[3,1] = self.truck_width - self.box_width
            
            # 比较货物是否与已有货物重叠
            for i in self.packed_output[self.truck_id]:
                if (self.packed_output[self.truck_id][i][2][0] > temp[0][0]) & (self.packed_output[self.truck_id][i][2][1] > temp[0][1]):
                    # 若重叠，也不能倒放，则结束
                    if self.box_length>self.truck_width:
                        return self.pack_tf, self.packed_output
                        break
                    # 若可以倒放，将货物的坐标倒放
                    else:
                        temp[:,0] = self.truck_length
                        temp[:,1] = self.truck_width
                        temp[:2,0] = self.truck_length - self.box_width
                        temp[0,1] = self.truck_width - self.box_length
                        temp[3,1] = self.truck_width - self.box_length
                        
                        # 再次比较货物是否与已有货物重叠
                        for j in self.packed_output[self.truck_id]:
                            if (self.packed_output[self.truck_id][j][2][0] > temp[0][0]) & (self.packed_output[self.truck_id][j][2][1] > temp[0][1]):
                                return self.pack_tf, self.packed_output
                                break
                        else:
                            
                                move_up = True
                                move_left = True
                                while move_left:
                                    # 向上移动优先
                                    while move_up:
                                        max_x = 0
                                        max_left = 0
                                        # 找到box上方最高的能够挡住box的货物，并向上移动
                                        for m in self.packed_output[self.truck_id]:
                                            if (self.packed_output[self.truck_id][m][2][1] > temp[0][1])&(self.packed_output[self.truck_id][m][3][1]<temp[1][1]):
                                                if self.packed_output[self.truck_id][m][2][0] > max_x:
                                                    max_x = self.packed_output[self.truck_id][m][2][0]
                                                    max_left = self.packed_output[self.truck_id][m][0][1]
                                        temp[:,0] - (temp[0][0] - max_x)
                                        
                                        move_up = False
                                    
                                    max_y = 0
                                    # 向左移动，找到能够挡住box左移的货物
                                    for n in self.packed_output[self.truck_id]:
                                        if self.packed_output[self.truck_id][n][2][0] > temp[0][0]& self.packed_output[self.truck_id][n][1][0]<temp[0][0]:
                                            if self.packed_output[self.truck_id][n][1][1] > max_y:
                                                max_y = self.packed_output[self.truck_id][n][1][1]
                                    # 如果box无法全部移动至左边，则移动到挡住box的货物右侧
                                    if (max_left - max_y) < self.box_width:
                                        temp[:,1] - (temp[0][1]-max_y)
                                    # 否则，全部移动到box上方货物的左侧
                                    else:
                                        temp[:,1] - (temp[1][1]-max_left)
                                    
                                    if temp[0][0] == 0:
                                        move_left = False
                                        self.packed_output[self.truck_id][box_id] = np.zeros((4,2))
                                        self.packed_output[self.truck_id][box_id] += temp
                                        self.packed_output['sum_weight'] += self.box_weight
                                        self.pack_tf = True
                                        return self.pack_tf, self.packed_output
                                        
                                    else:
                                        max_x = 0
                                        for l in self.packed_output[self.truck_id]:
                                            if (self.packed_output[self.truck_id][l][2][1] > temp[0][1])&(self.packed_output[self.truck_id][l][3][1]<temp[1][1]):
                                                if self.packed_output[self.truck_id][l][2][0] > max_x:
                                                    max_x = self.packed_output[self.truck_id][l][2][0]
                                        if max_x == temp[0][0]:
                                            move_left = False
                                            self.packed_output[self.truck_id][box_id] = np.zeros((4,2))
                                            self.packed_output[self.truck_id][box_id] += temp
                                            self.packed_output['sum_weight'] += self.box_weight
                                            self.pack_tf = True
                                            return self.pack_tf, self.packed_output
                                
            else:
                
                move_up = True
                move_left = True                
                while move_left:
                # 向上移动优先
                    while move_up:
                        max_x = 0
                        max_left = 0
                        # 找到box上方最高的能够挡住box的货物，并向上移动
                        for m in self.packed_output[self.truck_id]:
                            if (self.packed_output[self.truck_id][m][2][1] > temp[0][1])&(self.packed_output[self.truck_id][m][3][1]<temp[1][1]):
                                if self.packed_output[self.truck_id][m][2][0] > max_x:
                                    max_x = self.packed_output[self.truck_id][m][2][0]
                                    max_left = self.packed_output[self.truck_id][m][0][1]
                                    temp[:,0] - (temp[0][0] - max_x)
                                    move_up = False
                    max_y = 0
                    # 向左移动，找到能够挡住box左移的货物
                    for n in self.packed_output[self.truck_id]:
                        if self.packed_output[self.truck_id][n][2][0] > temp[0][0]& self.packed_output[self.truck_id][n][1][0]<temp[0][0]:
                            if self.packed_output[self.truck_id][n][1][1] > max_y:
                                max_y = self.packed_output[self.truck_id][n][1][1]
                                # 如果box无法全部移动至左边，则移动到挡住box的货物右侧
                                if (max_left - max_y) < self.box_width:
                                    temp[:,1] - (temp[0][1]-max_y)
                                    # 否则，全部移动到box上方货物的左侧
                                else:
                                    temp[:,1] - (temp[1][1]-max_left)
                                    
                    if temp[0][0] == 0:
                        move_left = False
                        self.packed_output[self.truck_id][box_id] = np.zeros((4,2))
                        self.packed_output[self.truck_id][box_id] += temp
                        self.packed_output['sum_weight'] += self.box_weight
                        self.pack_tf = True
                        return self.pack_tf, self.packed_output
                                        
                    else:
                        max_x = 0
                        for l in self.packed_output[self.truck_id]:
                            if (self.packed_output[self.truck_id][l][2][1] > temp[0][1])&(self.packed_output[self.truck_id][l][3][1]<temp[1][1]):
                                if self.packed_output[self.truck_id][l][2][0] > max_x:
                                    max_x = self.packed_output[self.truck_id][l][2][0]
                        if max_x == temp[0][0]:
                            move_left = False
                            self.packed_output[self.truck_id][box_id] = np.zeros((4,2))
                            self.packed_output[self.truck_id][box_id] += temp
                            self.packed_output['sum_weight'] += self.box_weight
                            self.pack_tf = True
                            return self.pack_tf, self.packed_output

In [35]:
test1 = BLpacking('C1', 20, 8, 100)

In [36]:
test1.packing('box1',2,1,5)

(True, {'C1': {'box1': array([[0., 0.],
          [0., 1.],
          [2., 1.],
          [2., 0.]])}, 'box_exit': True, 'sum_weight': 5})

In [40]:
test1.packing('box2',4,3,10)

KeyboardInterrupt: 

In [39]:
test1.box_weight

10